In [ ]:
import os
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,GlobalMaxPooling2D
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [ ]:
from zipfile import ZipFile
ZipFile("/content/drive/MyDrive/Project_CE434/car_moto_4k/test1.zip","r").extractall()
ZipFile("/content/drive/MyDrive/Project_CE434/car_moto_4k/training1.zip", "r").extractall()
ZipFile("/content/drive/MyDrive/Project_CE434/car_moto_4k/validate1.zip", "r").extractall()

In [ ]:
TRAIN_PATH = './training1'
TEST_PATH = './test1'
VALID_PATH = './validate1'
MODELS_PATH = '/content/drive/MyDrive/Project_CE434/car_moto_4k/model_final/'
VISUALIZE_PATH = '/content/drive/MyDrive/Project_CE434/car_moto_4k/visualize2/'
WIDTH = 112
HEIGHT = 112
EPOCHS = 50
BATCH_SIZE = 32

In [ ]:
train_generator = image.ImageDataGenerator(
    rescale = 1./255,  
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 15
)
test_generator = image.ImageDataGenerator(
    rescale = 1./255,    
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 15
)
valid_generator = image.ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 15
)

In [ ]:
generated_train_data = train_generator.flow_from_directory(
    TRAIN_PATH,
    target_size = (WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = True, 
    seed=42
)

Found 3600 images belonging to 2 classes.


In [ ]:
generated_valid_data = valid_generator.flow_from_directory(
    VALID_PATH,
    target_size = (WIDTH, HEIGHT),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = True,
    seed=42
)

Found 520 images belonging to 2 classes.


In [ ]:
generated_test_data = test_generator.flow_from_directory(
    TEST_PATH,
    target_size = (WIDTH, HEIGHT),
    class_mode = 'categorical',
    shuffle = False,
)

Found 900 images belonging to 2 classes.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()
model.add(Conv2D(name="block1_conv1",input_shape=(WIDTH, HEIGHT,3),filters=8,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(name="block1_conv2",filters=8,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(name="block2_conv1",filters=16, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block2_conv2",filters=16, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(name="block3_conv1",filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block3_conv2",filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block3_conv3",filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(name="block4_conv1",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block4_conv2",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block4_conv3",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(name="block5_conv1",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block5_conv2",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(name="block5_conv3",filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
# Dong bang cac layer
model.add(GlobalMaxPooling2D())
model.add(Dense(32,activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dense(2, activation="softmax"))


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(lr = 1e-5), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 112, 112, 8)       224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 8)       584       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 16)        2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 28, 28, 32)       

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
vgg16_history = model.fit(
    generated_train_data, 
    validation_data = generated_valid_data,
    validation_steps = generated_valid_data.n // BATCH_SIZE,
    steps_per_epoch = generated_train_data.n  // BATCH_SIZE,
    epochs = EPOCHS,
    verbose = 1
)
model.save(MODELS_PATH + 'model_trainVGG16.h5')
model.save_weights(MODELS_PATH + 'model_weight.h5')

Epoch 1/50
112/112 [==============================] - 83s 455ms/step - loss: 0.6928 - accuracy: 0.5000 - val_loss: 0.6923 - val_accuracy: 0.5020
Epoch 2/50
112/112 [==============================] - 48s 431ms/step - loss: 0.6912 - accuracy: 0.5053 - val_loss: 0.6902 - val_accuracy: 0.5020
Epoch 3/50
112/112 [==============================] - 49s 435ms/step - loss: 0.6857 - accuracy: 0.5062 - val_loss: 0.6821 - val_accuracy: 0.5254
Epoch 4/50
112/112 [==============================] - 49s 438ms/step - loss: 0.6676 - accuracy: 0.5959 - val_loss: 0.6544 - val_accuracy: 0.6562
Epoch 5/50
112/112 [==============================] - 49s 442ms/step - loss: 0.6244 - accuracy: 0.6892 - val_loss: 0.6052 - val_accuracy: 0.6719
Epoch 6/50
112/112 [==============================] - 50s 446ms/step - loss: 0.5622 - accuracy: 0.7281 - val_loss: 0.5576 - val_accuracy: 0.7285
Epoch 7/50
112/112 [==============================] - 49s 437ms/step - loss: 0.5272 - accuracy: 0.7413 - val_loss: 0.5303 - val_ac

In [ ]:
y_test = generated_test_data.classes
y_pred_vgg16 = model.predict(generated_test_data, verbose = 1)

29/29 [==============================] - 12s 385ms/step


In [ ]:
generated_train_data.class_indices.items()

dict_items([('car', 0), ('moto', 1)])

In [ ]:
df = pd.DataFrame({
    'Actual'     : generated_test_data.filenames, 
    'VGG16 prediction' : np.argmax(y_pred_vgg16, axis=1)
})  
df.head(20)

,Actual,VGG16 prediction
0,car/car.1.jpg,0
1,car/car.10.jpg,0
2,car/car.100.jpg,0
3,car/car.101.jpg,0
4,car/car.102.jpg,0
5,car/car.103.jpg,0
6,car/car.104.jpg,0
7,car/car.105.jpg,0
8,car/car.106.jpg,0
9,car/car.107.jpg,0


In [ ]:
df.tail(20)

,Actual,VGG16 prediction
880,moto/moto.81.jpg,1
881,moto/moto.82.jpg,1
882,moto/moto.83.jpg,1
883,moto/moto.84.jpg,1
884,moto/moto.85.jpg,1
885,moto/moto.86.jpg,1
886,moto/moto.87.jpg,1
887,moto/moto.88.jpg,1
888,moto/moto.89.jpg,1
889,moto/moto.9.jpg,1


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns

cf_matrix_vgg16 = confusion_matrix(y_test, np.argmax(y_pred_vgg16, axis=1))
print(classification_report(y_test, np.argmax(y_pred_vgg16, axis=1)))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91       450
           1       0.93      0.88      0.91       450

    accuracy                           0.91       900
   macro avg       0.91      0.91      0.91       900
weighted avg       0.91      0.91      0.91       900



In [ ]:
import h5py
import numpy as np
from keras.models import load_model

model_done = load_model(MODELS_PATH + 'model_trainVGG16.h5')
# model_done.summary()
cnt = 0
for layer in model_done.layers:
 

  # check for convolutional layer
  if 'conv' not in layer.name:
    continue
  # get filter weights
  filters, biases = layer.get_weights()
  print(layer.name, filters.shape)
  print(len(filters), len(biases))

block1_conv1 (3, 3, 3, 8)
3 8
block1_conv2 (3, 3, 8, 8)
3 8
block2_conv1 (3, 3, 8, 16)
3 16
block2_conv2 (3, 3, 16, 16)
3 16
block3_conv1 (3, 3, 16, 32)
3 32
block3_conv2 (3, 3, 32, 32)
3 32
block3_conv3 (3, 3, 32, 32)
3 32
block4_conv1 (3, 3, 32, 64)
3 64
block4_conv2 (3, 3, 64, 64)
3 64
block4_conv3 (3, 3, 64, 64)
3 64
block5_conv1 (3, 3, 64, 64)
3 64
block5_conv2 (3, 3, 64, 64)
3 64
block5_conv3 (3, 3, 64, 64)
3 64
